# OPTaaS Batching

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:charles.brecque@mindfoundry.ai">here</a>.

OPTaaS can facilitate parallel computation, where you generate a batch of configurations, pass them to a number of workers to calculate the results, and then store the results to get the next batch of configurations.

## Connect to OPTaaS using your API Key

In [1]:
library(optaas.client)

client <- OPTaaSClient$new("https://optaas.mindfoundry.ai", "Your OPTaaS API Key")

## Create your Task

In [2]:
task <- client$create_task(
    title="Batching Example",
    parameters=list(
        FloatParameter('x', minimum = 0, maximum = 1),
        FloatParameter('y', minimum = 0.1, maximum = 2)
    )
)

## Set up your workers
This is just a simple example of how you would pass a Configuration to a worker and get a Result back. Your process will of course likely be more complex!

In [3]:
library(parallel)

number_of_workers <- 4

scoring_function <- function(configuration) {
    x <- configuration$values[["x"]]
    y <- configuration$values[["y"]]
    Sys.sleep(1)  # Calculating score...
    score <- (x * y) - (x / y)
    list(configuration=configuration$id, score=score)
}

spin_off_workers_and_get_results <- function(configurations) {
    mclapply(configurations, scoring_function)
}

## Generate the first batch of configurations

In [4]:
configurations <- task$generate_configurations(number_of_workers)
print(paste(lapply(configurations, `[[`, "values")))

[1] "list(x = 0.5, y = 1.05)"     "list(x = 0.75, y = 0.575)"  
[3] "list(x = 0.25, y = 1.525)"   "list(x = 0.375, y = 0.8125)"


## Record results to get the next batch of configurations

The next batch will be the same size as the number of results you record.

In [5]:
for (batch in seq(1, 3)) {
    results <- spin_off_workers_and_get_results(configurations)
    print(paste("Batch", batch))
    print(paste(results))
    flush.console()

    configurations <- task$record_results(results)
}

[1] "Batch 1"
[1] "list(configuration = \"025151df-4967-4949-8cf9-ebd744d5cd24\", score = 0.0488095238095239)"
[2] "list(configuration = \"ec7ab627-455f-42d3-bd2b-45cd3e307d43\", score = -0.873097826086957)"
[3] "list(configuration = \"4bfe6632-7599-4fc0-aada-21da7a98c6d0\", score = 0.217315573770492)" 
[4] "list(configuration = \"564244e9-ebd3-4bde-a403-6c28767ae4c6\", score = -0.156850961538462)"
[1] "Batch 2"
[1] "list(configuration = \"d3b0d0b8-d4f5-45ba-9a57-b395572dcc6b\", score = 1.04573359929078)"  
[2] "list(configuration = \"96166ed5-dcac-4d48-a582-152dc5c6bb95\", score = -1.64091435185185)" 
[3] "list(configuration = \"425d6ce4-50be-4d25-982d-8f1032b98d0a\", score = 0.0638501213592233)"
[4] "list(configuration = \"aa9460ee-385c-48cb-a57f-70cda8d75bff\", score = -0.14019214527027)" 
[1] "Batch 3"
[1] "list(configuration = \"c2419ee9-c899-4428-b973-198ed37cd84a\", score = 0.711827174429658)"
[2] "list(configuration = \"764d6065-459a-46b7-a1ea-8c6afd09cd40\", score = -4.0806361